In [ ]:
from advsecurenet.attacks.gradient_based import FGSM, LOTS, PGD
from advsecurenet.shared.types.configs.attack_configs import (
    FgsmAttackConfig,
    LotsAttackConfig,
    PgdAttackConfig,
)
from advsecurenet.shared.types.configs.attack_configs.attacker_config import (
    AttackerConfig,
)
from advsecurenet.models.model_factory import ModelFactory
from advsecurenet.datasets.dataset_factory import DatasetFactory
from advsecurenet.attacks.attacker import Attacker
from advsecurenet.dataloader.data_loader_factory import DataLoaderFactory
from advsecurenet.shared.types.configs.preprocess_config import (
    PreprocessConfig,
    PreprocessStep,
)
from advsecurenet.shared.types.configs.device_config import DeviceConfig
from advsecurenet.utils.adversarial_target_generator import AdversarialTargetGenerator
from advsecurenet.datasets.targeted_adv_dataset import AdversarialDataset
from advsecurenet.defenses.adversarial_training import AdversarialTraining
from advsecurenet.shared.types.configs.defense_configs.adversarial_training_config import (
    AdversarialTrainingConfig,
)

In [ ]:
# Define the model
model = ModelFactory.create_model(
    model_name="resnet18", num_classes=10, pretrained=True
)

In [ ]:
# Lets define the preprocessing configuration we want to use
preprocess_config = PreprocessConfig(
    steps=[
        PreprocessStep(name="Resize", params={"size": 32}),
        PreprocessStep(name="CenterCrop", params={"size": 32}),
        PreprocessStep(name="ToTensor"),
        PreprocessStep(
            name="ToDtype", params={"dtype": "torch.float32", "scale": True}
        ),
        PreprocessStep(
            name="Normalize",
            params={"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]},
        ),
    ]
)

# Define the dataset
dataset = DatasetFactory.create_dataset(
    dataset_type="cifar10", preprocess_config=preprocess_config, return_loaded=False
)
train_data = dataset.load_dataset(train=True)
test_data = dataset.load_dataset(train=False)

In [ ]:
# Define the dataloder
dataloader = DataLoaderFactory.create_dataloader(dataset=train_data, batch_size=32)

In [ ]:
# define the device config
device = DeviceConfig(processor="mps")

# Define the fgsm config
fgsm_config = FgsmAttackConfig(
    targeted=False,
    epsilon=0.1,
    device=device,
)

# Now we can define the attack
fgsm_attack = FGSM(config=fgsm_config)

In [ ]:
pgd_config = PgdAttackConfig(
    targeted=False,
    epsilon=0.1,
    alpha=0.01,
    num_iter=10,
    device=device,
)

pgd_attack = PGD(config=pgd_config)

In [ ]:
adversarial_training_config = AdversarialTrainingConfig(
    model=model,
    models=[],  # no ensemble of models
    attacks=[fgsm_attack, pgd_attack],  # ensemble of attacks
    processor=device.processor,
    train_loader=dataloader,
    epochs=2,
)

adversarial_training = AdversarialTraining(config=adversarial_training_config)

adversarial_training.train()

We can aslo combine targeted and non-targeted attacks. In this case, we need to either provide the targets or let the `advsecurenet` to generate them. And finally, we need to use `AdversarialDataset` as the dataset.

In [ ]:
# create adversarial target generator
target_generator = AdversarialTargetGenerator()

# We need to generate target images and labels for the targeted attack
target_images, target_labels = target_generator.generate_target_images_and_labels(
    data=train_data, overwrite=True
)

# Again lets create a new dataset with the target images and labels
targeted_dataset = AdversarialDataset(
    base_dataset=train_data,
    target_images=target_images,
    target_labels=target_labels,
)

# And a new dataloader
targeted_dataloader = DataLoaderFactory.create_dataloader(
    dataset=targeted_dataset, batch_size=32
)

In [ ]:
targeted_fgsm_config = FgsmAttackConfig(
    targeted=True,
    epsilon=0.1,
    device=device,
)

targeted_fgsm = FGSM(config=targeted_fgsm_config)

It is also possible to use ensemble of models. If we use ensemble of models, in each batch one of the models is selected randomly and the attack is performed on that model.

In [ ]:
vgg16 = ModelFactory.create_model(model_name="vgg16", num_classes=10, pretrained=False)

In [ ]:
# Here we are using the resnet18 as the main model that we aim to defend and the vgg16 as the model that we use to generate the adversarial examples
# Also we are using 3 attacks, the fgsm, pgd and targeted_fgsm
targeted_adversarial_training_config = AdversarialTrainingConfig(
    model=model,
    models=[vgg16],  # no ensemble of models
    attacks=[fgsm_attack, pgd_attack, targeted_fgsm],  # ensemble of attacks
    processor=device.processor,
    train_loader=targeted_dataloader,
    epochs=1,
)

adversarial_training = AdversarialTraining(config=targeted_adversarial_training_config)

adversarial_training.train()